# Classification Models
- Models to be used:  
    -K-Nearest Neighbors  
    -Random Forest  
    -Random Forest with Gridsearch  
    -Naive Bayes  

In [1]:
%load_ext autoreload
%autoreload 2
 
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import re

import time

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.preprocessing import StandardScaler, label_binarize, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import sklearn.datasets as datasets

import pandas_profiling

from sklearn.linear_model import LogisticRegression, LinearRegression
# import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv('./Datasets/clean_train.csv')
df_test = pd.read_csv('./Datasets/Tan_Test.csv')

In [5]:
df.head()

,id,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,region_code,district_code,...,date_recordedDay,date_recordedDayofweek,date_recordedDayofyear,date_recordedIs_month_end,date_recordedIs_month_start,date_recordedIs_quarter_end,date_recordedIs_quarter_start,date_recordedIs_year_end,date_recordedIs_year_start,date_recordedElapsed
0,69572,6000.0,1390,34.938093,-9.856322,0,Lake Nyasa,Iringa,11,5,...,14,0,73,False,False,False,False,False,False,3073
1,8776,0.0,1399,34.698766,-2.147466,0,Lake Victoria,Mara,20,2,...,6,2,65,False,False,False,False,False,False,3796
2,34310,25.0,686,37.460664,-3.821329,0,Pangani,Manyara,21,4,...,25,0,56,False,False,False,False,False,False,3787
3,67743,0.0,263,38.486161,-11.155298,0,Ruvuma / Southern Coast,Mtwara,90,63,...,28,0,28,False,False,False,False,False,False,3759
4,19728,0.0,0,31.130847,-1.825359,0,Lake Victoria,Kagera,18,1,...,13,2,194,False,False,False,False,False,False,3194


In [6]:
df.columns

Index(['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude',
       'num_private', 'basin', 'region', 'region_code', 'district_code',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'status_group',
       'date_recordedYear', 'date_recordedMonth', 'date_recordedWeek',
       'date_recordedDay', 'date_recordedDayofweek', 'date_recordedDayofyear',
       'date_recordedIs_month_end', 'date_recordedIs_month_start',
       'date_recordedIs_quarter_end', 'date_recordedIs_quarter_start',
       'date_recordedIs_year_end', 'date_recordedIs_year_start',
       'date_recordedElapsed'],
      dtype='object')

In [11]:
df_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
